In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#자료 불러오기
app_store_df = pd.read_csv("./app_store.csv")

#자료 중 Price, Size, Installs를 숫자 형태로 변환
app_store_df['NPriceD'] = app_store_df.Price.map(lambda x : float(x[1:]))
app_store_df['NSizeM'] = app_store_df.Size.map(lambda x : float(x.rstrip('M').replace(',','')) if(x[-1] == 'M') else float(x.rstrip('k').replace(',',''))/1000)
app_store_df['Installs'] = app_store_df.Installs.map(lambda x : int(x.rstrip('+').replace(',','')))

#자료 앞부분 출력
app_store_df.head()

,App Name,Category,Rating,Reviews,Installs,Size,Price,NPriceD,NSizeM
0,123 Kids Fun MUSIC - Kids Music Educational Games,GAME_MUSIC,4.083333,36,1000,22M,$1.99,1.99,22.0
1,LetterSchool - Block Letters,GAME_EDUCATIONAL,4.181092,751,10000,79M,$6.99,6.99,79.0
2,Tynker for Schools,GAME_EDUCATIONAL,4.000000,33,1000,33M,$4.99,4.99,33.0
3,Fuzzy Numbers: Pre-K Number Foundation,EDUCATION,4.500000,22,1000,44M,$5.99,5.99,44.0
4,Toddler Sing and Play 2 Pro,GAME_MUSIC,4.065041,246,10000,20M,$1.99,1.99,20.0


In [11]:
#Selenium 라이브러리를 활용하여 플레이스토어 크롤링 환경 구성
from selenium import webdriver
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.chrome.options import Options 
import time
start_time = time.time()

#headless Chrome Canary를 이용하여 크롤링
chrome_options = Options()  
chrome_options.add_argument("--headless")  
chrome_options.binary_location = '/Applications/Google Chrome Canary.app/Contents/MacOS/Google Chrome Canary'

#웹사이트 접속 테스트
driver = webdriver.Chrome(executable_path=os.path.abspath("chromedriver"),   chrome_options=chrome_options)  
driver.get('https://play.google.com/store/apps/details?id=com.snkplaymore.android001')
rating_element = driver.find_element_by_class_name('VEF2C').get_attribute('innerHTML')

print(rating_element)


print("--- %s seconds ---" % (time.time() - start_time))

<div class="mMF0fd"><span class="Gn2mNd">5</span><span class="L2o20d P41RMc" style="width: 100%"></span></div><div class="mMF0fd"><span class="Gn2mNd">4</span><span class="L2o20d tpbQF" style="width: 16%"></span></div><div class="mMF0fd"><span class="Gn2mNd">3</span><span class="L2o20d Sthl9e" style="width: 6%"></span></div><div class="mMF0fd"><span class="Gn2mNd">2</span><span class="L2o20d rhCabb" style="width: 3%"></span></div><div class="mMF0fd"><span class="Gn2mNd">1</span><span class="L2o20d A3ihhc" style="width: 10%"></span></div>
--- 5.26319694519043 seconds ---


In [23]:
#각 어플의 플레이스토어 존재 여부 확인 및 package name 저장
import time
start_time = time.time()

#플레이스토어에 이름을 검색한 뒤 상위 5개 항목에 이름과 일치하는 항목이 있을 경우 존재한다고 판단하고 이후 크롤링을 위해 package name 저장
#아닐 경우 존재하지 않는다고 판단하고 데이터에서 제거 
package_names = pd.Series()
for i in list(range(len(app_store_df['App Name']))):
    driver.get('https://play.google.com/store/search?q='+app_store_df['App Name'][i]+'&c=apps')
    for  j in [0,1,2,3,4,5]:        #상위 6개 항목 확인
        try:
            #app name 저장 indicator - WsMG1c nnK0zc 클래스
            app_name = driver.find_elements_by_css_selector(".WsMG1c.nnK0zc")[j].get_attribute('innerHTML')
            print(app_name) #어플 이름 검색 결과 로그
            if (app_store_df['App Name'][i] == app_name):
                #package name 저장 indicator - vU6FJ p63iDd 클래스의 a 태그
                search_div = driver.find_elements_by_css_selector(".vU6FJ.p63iDd a")[0].get_attribute('href')   
                print(search_div) #package name이 포함되어있는 해당 어플의 url 로그
                package_name = search_div[46:]
                package_names.at[i] = package_name
                break
            else:
                package_names.at[i] = 'Not in store'
        except:
            pass
print(package_names)
    

print("--- %s seconds ---" % (time.time() - start_time)) #오래 걸리는 작업이므로 시간 로그

Concrete, Steel, Bricks)
YoWindow - best weather app with live pictures
YoWindow Weather - Unlimited
Weather &amp; Widget - Weawow
🌈Weather Live Wallpapers
Accurate Weather - Live Weather Forecast
Animated 3D Weather
Dish Install, Pointer &amp; Align
Set Satellite Dish 2020
DishPointer (Satellite Finder &amp; dish installer)
Satellite Pointer
Satellite Finder (Dish Aligner)
Satellite Finder - Dish Pointer
Ultimate Farm Simulator
https://play.google.com/store/apps/details?id=com.glutenfreegames.ultimatefarmsimulator
Thermal Camera For FLIR One
Thermal Camera+ for FLIR One
https://play.google.com/store/apps/details?id=georg.com.flironetest_01
Photographer's companion Pro
https://play.google.com/store/apps/details?id=com.stefsoftware.android.photographerscompanionpro
Bose SoundTouch
Sound Touch 2
Sound Touch
https://play.google.com/store/apps/details?id=com.bose.soundtouch
Video Touch - Vehicles
https://play.google.com/store/apps/details?id=com.vehiclesvideotouch
Video Touch - Wildlife
ht

In [75]:
#플레이스토어에 없는 어플 데이터에서 제거
app_store_df['package_name'] = package_names
app_store_df = app_store_df[app_store_df['package_name'] != 'Not in store'].reset_index(drop=True)
app_store_df

,App Name,Category,Rating,Reviews,Installs,Size,Price,NPriceD,NSizeM,package_name
0,Fuzzy Numbers: Pre-K Number Foundation,EDUCATION,4.500000,22,1000,44M,$5.99,5.99,44.0,com.cognitivetoybox.fuzzynumbers
1,Toddler Sing and Play 2 Pro,GAME_MUSIC,4.065041,246,10000,20M,$1.99,1.99,20.0,com.landoncope.games.toddlersingandplay2
2,Green Eggs and Ham - Dr. Seuss,BOOKS_AND_REFERENCE,4.656376,745,10000,14M,$3.99,3.99,14.0,com.oceanhouse_media.bookgreeneggsandham_app
3,Meme Generator PRO,ENTERTAINMENT,4.630876,4042,100000,58M,$2.99,2.99,58.0,com.zombodroid.MemeGeneratorFullGplay
4,Construction Simulator 2014,GAME_SIMULATION,4.339837,104091,1000000,19M,$0.99,0.99,19.0,com.astragon.cs2014
...,...,...,...,...,...,...,...,...,...,...
1196,Infinite Dose PRO: The Smart Dosage Calculator,MEDICAL,4.764102,195,1000,3.7M,$1.99,1.99,3.7,com.moaness.InfiniteDose.free
1197,Bartender's Choice Vol.2,FOOD_AND_DRINK,3.958333,24,1000,40M,$4.99,4.99,40.0,com.bartenderschoice
1198,Musicana Pro Music Player,MUSIC_AND_AUDIO,4.030769,65,500,10M,$1.49,1.49,10.0,com.musicplayer.musicana.pro
1199,Fella for Facebook,SOCIAL,4.256288,4811,50000,2.3M,$0.99,0.99,2.3,fellasocial.app


In [76]:
#Rating 도수분포의 표준편차 구하는 함수 작성
def std_rating(ratings):
    list_to_std = []
    for i in list(range(5)):
        for j in range(ratings[i]):
            list_to_std.append(5-i)
    return np.std(list_to_std)

In [81]:
#각 어플 Rating의 표준편차 구하기 - Play Store 어플 웹페이지 내에 있는 바 차트 활용
stds_rating = pd.Series()
for i in list(range(len(app_store_df))):
    try:
        driver.get('https://play.google.com/store/apps/details?id='+app_store_df['package_name'][i])
        ratings = []
        ratings.append(int(driver.find_elements_by_css_selector(".L2o20d.P41RMc")[0].get_attribute('style')[7:][:-2]))
        ratings.append(int(driver.find_elements_by_css_selector(".L2o20d.tpbQF")[0].get_attribute('style')[7:][:-2]))
        ratings.append(int(driver.find_elements_by_css_selector(".L2o20d.Sthl9e")[0].get_attribute('style')[7:][:-2]))
        ratings.append(int(driver.find_elements_by_css_selector(".L2o20d.rhCabb")[0].get_attribute('style')[7:][:-2]))
        ratings.append(int(driver.find_elements_by_css_selector(".L2o20d.A3ihhc")[0].get_attribute('style')[7:][:-2]))
        print(app_store_df['App Name'][i])
        print(std_rating(ratings))
        stds_rating.at[i] = std_rating(ratings)
    except:
        stds_rating.at[i] = float('nan')


5947205966737583
Motor Depot
1.2063613456543658
SkySafari 6 Pro
0.9205498951034647
SkySafari 6 Plus
0.9665303393990254
Wine Tasting
1.5512686916498744
Latin Chords (LaCuerda PRO)
0.9367467577205426
Plague Inc: Scenario Creator
1.4042359312765234
Crossty
1.8894803812256262
eCanvas for cross-stitch PRO
1.4727901571084092
X-Stitch
1.3737145718560182
Clipboard Manager Pro
1.7826317417507211
Clipboard Pro
1.0681939189762597
Season Zen HD
1.0812308803762796
Oddworld: Munch's Oddysee
0.9199205955870451
Jet Car Stunts
1.3614239454067316
Budget Bytes - Delicious Recipes for Small Budgets
1.0170422803403996
Camera 4K Pro - Perfect, Selfie, Video, Photo
1.6389562794505574
Wild Tank | Pro Edition
1.7512589543459336
DiskDigger Pro file recovery
1.8836164614742832
S Launcher Pro for Galaxy
1.4654482198245848
My PlayHome Hospital
1.2452812863758962
G-Stomper VA-Beast Synthesizer
0.6474945414086076
Rescue the Enchanter
0.9849337078985151
USB Audio Recorder PRO
1.7354960758929179
USB Audio Player PRO
1

In [90]:
#app_store_df에 표준편차 column 추가 후 파일로 저장 및 확인출력
app_store_df['stds_rating'] = stds_rating
app_store_df.to_csv('app_store_modified_2.csv')
app_store_df

,App Name,Category,Rating,Reviews,Installs,Size,Price,NPriceD,NSizeM,package_name,stds_rating
0,Fuzzy Numbers: Pre-K Number Foundation,EDUCATION,4.500000,22,1000,44M,$5.99,5.99,44.0,com.cognitivetoybox.fuzzynumbers,0.977582
1,Toddler Sing and Play 2 Pro,GAME_MUSIC,4.065041,246,10000,20M,$1.99,1.99,20.0,com.landoncope.games.toddlersingandplay2,1.029111
2,Green Eggs and Ham - Dr. Seuss,BOOKS_AND_REFERENCE,4.656376,745,10000,14M,$3.99,3.99,14.0,com.oceanhouse_media.bookgreeneggsandham_app,0.932432
3,Meme Generator PRO,ENTERTAINMENT,4.630876,4042,100000,58M,$2.99,2.99,58.0,com.zombodroid.MemeGeneratorFullGplay,0.635199
4,Construction Simulator 2014,GAME_SIMULATION,4.339837,104091,1000000,19M,$0.99,0.99,19.0,com.astragon.cs2014,1.225161
...,...,...,...,...,...,...,...,...,...,...,...
1196,Infinite Dose PRO: The Smart Dosage Calculator,MEDICAL,4.764102,195,1000,3.7M,$1.99,1.99,3.7,com.moaness.InfiniteDose.free,1.243694
1197,Bartender's Choice Vol.2,FOOD_AND_DRINK,3.958333,24,1000,40M,$4.99,4.99,40.0,com.bartenderschoice,1.378311
1198,Musicana Pro Music Player,MUSIC_AND_AUDIO,4.030769,65,500,10M,$1.49,1.49,10.0,com.musicplayer.musicana.pro,1.653320
1199,Fella for Facebook,SOCIAL,4.256288,4811,50000,2.3M,$0.99,0.99,2.3,fellasocial.app,1.328676
